In [1]:
#%load_ext autoreload
#%autoreload 2

#import sys
#sys.path.insert(0,'..')

import os

import backtrader as bt
import backtrader.indicators as btind
import numpy as np
import scipy.signal as signal

from btgym import BTgymEnv, BTgymStrategy, BTgymDataset

from launcher import Launcher

In [ ]:
# GYM TEST ENV:
cluster_config = dict(
    host='127.0.0.1',
    port=12222,
    num_workers=8,
    num_ps=1,
    log_dir='./tmp/a3c_testing_gym',
)

env_config = dict(
    gym_id='Breakout-v0'
)


launcher = Launcher(
    cluster_config=cluster_config,
    env_config=env_config,
    train_steps=500000000,
    opt_learn_rate=1e-4,
    rollout_length=30,
    test_mode=True,
    model_summary_freq=50,
    episode_summary_freq=2,
    env_render_freq=10,
    verbose=0
    
)

In [ ]:
launcher.cluster_spec

In [ ]:
launcher.run()


In [27]:
class MyStrategy(BTgymStrategy):
    """
    Example subclass of BT server inner computation startegy.
    """
    
    def set_datalines(self):
        self.state = np.zeros(self.p.state_shape)
        
        
    def get_state(self):
        """
        Computes featurized RL-ready environment observation state
        by applying continious wavelet transform to time-embedded vector
        of close-price gradients.
        """
        # Z-normalize raw inputs:
        mean = self.p.dataset_stat[1:3].values[0,:-1]
        std = self.p.dataset_stat[1:3].values[1,:-1]
        X = (self.raw_state - mean) / std
        #X = self.raw_state
        
        # Prepare parameters:
        Tau = 5
        max_cwt_scale = self.p.state_shape[1]
        cwt_width = np.linspace(Tau, max_cwt_scale + Tau - 1, max_cwt_scale) # scale of wavelet transdorm [n]
        
        sigmoid = lambda x: 1/(1 + np.exp(-x))
        
        # 'gamma'-like signal hyperparameter
        # for our signal to be in about [-5,+5] range before passing it to sigmoid;
        # tweak it by hand to add/remove "contrast":
        T = 2.5e+2
        
        # Get vector of gradients of last [n] prices:
        X = np.gradient(X, axis=0) * T
        
        # Use close price:
        channel = 3
        # Compute continious wavelet transform using Ricker wavelet, get [n,m,1]-dim. matrix:
        self.state = signal.cwt(X[:, channel], signal.ricker, cwt_width).T[:, :, None]
        
        # Squash values in [0,1]:
        return sigmoid(self.state)
    

    def R(self, val, val_start, R_max, DD_max):
        """
        Piecewise-linear normalized reward:
        val > 0         - values scalar or vector;
        val_start > 1   - start value, scalar;
        0 < R_max < 1   - maximum gain, percent/100,
        0 < DD_max < 1  - maximum draw-down, percent/100,
                          scalar (minimal acceptable value: = val_start*(1-DD_max) );
        returns vector in [-1,+1] range :
        """

        f_neg = np.asarray((val/val_start + DD_max - 1) / DD_max - 1)
        f_neg[f_neg > 0] = 0 
        f_neg[f_neg < -1] = -0.999
        f_pos = np.asarray((val/val_start - 1) / R_max)
        f_pos[f_pos < 0] = 0
        f_pos[f_pos > 1] = 1 

        return (f_neg + f_pos) 
    
    def get_reward(self):
        """
        Defines reward as [0,1]-bounded linear function of current to initial portfolio value ratio.
        """
        r = float(
            self.R(
                val=self.env.broker.get_value(),
                val_start=self.env.broker.startingcash,
                R_max=self.p.target_call/100,
                DD_max=self.p.drawdown_call/100,
            )
        )/2 +.5  # shift reward to [0,1] range
        
        return r / 40
            
# Set backtesting engine parameters:

state_shape = (60, 30, 1)

MyCerebro = bt.Cerebro()
MyCerebro.addstrategy(MyStrategy,
                      state_shape=state_shape,
                      portfolio_actions=('hold', 'buy', 'sell'),
                      drawdown_call=5, # in percent
                      target_call=5,
                      state_low=0,
                      state_high=1,
                      skip_frame=2,
                     )

# Set leveraged account:
MyCerebro.broker.setcash(2000)
MyCerebro.broker.setcommission(commission=0.0001, leverage=10.0)
MyCerebro.broker.set_shortcash(False)
MyCerebro.addsizer(bt.sizers.SizerFix, stake=10000,)


MyCerebro.addanalyzer(bt.analyzers.DrawDown)

# Provide data:
MyDataset = BTgymDataset(
    #filename='../data/DAT_ASCII_EURUSD_M1_2016.csv',
    filename='../data/test_sine_1min_period256_delta0002.csv',
    start_weekdays=[0, 1, 2, 3, 4],
    episode_len_days=0,
    episode_len_hours=23,
    episode_len_minutes=0,
    start_00=False,
    time_gap_hours=2,
)
env_config = dict(
    dataset=MyDataset,
    engine=MyCerebro,
    render_agent_as_image=True,
    render_ylabel='Z-norm/CWT',
    render_size_episode=(12,8),
    render_size_human=(8, 3.5),
    render_size_agent=(10, 5),
    render_dpi=75,
    port=5000,
    data_port=4999,
    connect_timeout=60,
    verbose=0,
)
cluster_config = dict(
    host='127.0.0.1',
    port=12222,
    num_workers=8,
    num_ps=1,
    log_dir='./tmp/a3c_testing',
)
launcher = Launcher(
    cluster_config=cluster_config,
    env_class=BTgymEnv,
    env_config=env_config,
    rollout_length=120,
    test_mode=False,
    train_steps=1000000000,
    model_summary_freq=10,
    episode_summary_freq=2,
    env_render_freq=10,
    verbose=1
    
)

[2017-08-18 00:32:14,176] ./tmp/a3c_testing created.


In [28]:
launcher.run()

[2017-08-18 00:32:27,651] Press `Ctrl-C` to stop training and close launcher.


Press `Ctrl-C` to stop training and close launcher.
INFO:tensorflow:Starting queue runners.


[2017-08-18 00:32:35,607] Starting queue runners.
[2017-08-18 00:32:35,660] worker_1: starting training at step: 0


INFO:tensorflow:Starting queue runners.


[2017-08-18 00:32:35,889] Starting queue runners.


...

[2017-08-18 01:42:35,928] Saving checkpoint to path ./tmp/a3c_testing/train/model.ckpt


INFO:tensorflow:Saving checkpoint to path ./tmp/a3c_testing/train/model.ckpt


[2017-08-18 01:47:35,923] Saving checkpoint to path ./tmp/a3c_testing/train/model.ckpt
Process BTgymServer-111:1:
Process BTgymDataFeedServer-110:1:
Process BTgymServer-112:1:
Process BTgymServer-117:1:
Process BTgymServer-115:1:
Process BTgymServer-114:1:
[2017-08-18 01:50:18,551] worker_1 has joined.
Process BTgymServer-116:1:
Process BTgymServer-110:2:
Process BTgymServer-113:1:
[2017-08-18 01:50:18,558] worker_2 has joined.
[2017-08-18 01:50:18,560] worker_3 has joined.
[2017-08-18 01:50:18,573] worker_4 has joined.
Traceback (most recent call last):
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/muzikin/Yandex.Disk.localized/work/btgym/btgym/dataserver.py", line 100, in run
    service_input = socket.recv_pyobj()
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/site-packages/zmq/sugar/socket.py", line 491, in recv_pyobj
    msg = self.recv(flags)
  File "zmq/backend/cyt

  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/site-packages/backtrader/strategy.py", line 327, in _next
    self._next_analyzers(minperstatus)
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/site-packages/backtrader/cerebro.py", line 1564, in _runnext
    strat._next()
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/site-packages/backtrader/strategy.py", line 361, in _next_analyzers
    analyzer._next()
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/site-packages/backtrader/strategy.py", line 361, in _next_analyzers
    analyzer._next()
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/site-packages/backtrader/strategy.py", line 327, in _next
    self._next_analyzers(minperstatus)
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/site-packages/backtrader/strategy.py", line 327, in _next
    self._next_analyzers(minperstatus)
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/site-packag

  File "zmq/backend/cython/socket.pyx", line 727, in zmq.backend.cython.socket.Socket.recv (zmq/backend/cython/socket.c:7460)
  File "zmq/backend/cython/socket.pyx", line 145, in zmq.backend.cython.socket._recv_copy (zmq/backend/cython/socket.c:2344)
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/site-packages/scipy/signal/wavelets.py", line 362, in cwt
    wavelet_data = wavelet(min(10 * width, len(data)), width)
  File "zmq/backend/cython/checkrc.pxd", line 12, in zmq.backend.cython.checkrc._check_rc (zmq/backend/cython/socket.c:9621)
KeyboardInterrupt
  File "zmq/backend/cython/socket.pyx", line 145, in zmq.backend.cython.socket._recv_copy (zmq/backend/cython/socket.c:2344)
KeyboardInterrupt
  File "zmq/backend/cython/socket.pyx", line 145, in zmq.backend.cython.socket._recv_copy (zmq/backend/cython/socket.c:2344)
KeyboardInterrupt
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/site-packages/scipy/signal/wavelets.py", line 306, in ricker
    gauss = n

In [ ]:
print(launcher.kwargs, '\n\n')
print(launcher.env_config)
print(launcher.cluster_config)
print(launcher.cluster_spec)
for config in launcher.workers_config_list:
    print('============')
    for k, v in config.items():
        print('{}:\n{}\n'.format(k, v))
    

In [ ]:
def func1(max_step):
    step = 0
    done = False
    
    def func2(max_step):
        nonlocal step
        nonlocal done
        step +=1
        if step == max_step:
            step = 0
            done = True
        return step
    
    for i in range(20):
        done = False
        print(func2(max_step), step, done)
        


func1(7)
            

In [ ]:
a = dict()
a.update({'b': 2, 'c':4})
type(a) == dict